In [21]:
import os

import pandas as pd

from usa_spent.extract import extract
from usa_spent.transform import get_transaction_type

In [2]:
data_path = os.path.join(os.path.realpath('.'), 'data')

In [3]:
if not os.path.exists(data_path):
    os.mkdir(data_path)

In [5]:
file_loc = os.path.join(data_path, 'output1.csv')

#Extract

In [6]:
extract('2017-03-01', '2017-03-01', file_loc)

Searching params: {'filters': [{'field': 'action_date', 'operation': 'less_than_or_equal', 'value': '2017-03-01'}, {'field': 'action_date', 'operation': 'greater_than_or_equal', 'value': '2017-03-01'}]}


Completed data pull! 8765 records found


#Transform

In [10]:
df = pd.read_csv(file_loc, low_memory=False)

In [18]:
df = df[sorted(list(df.columns))]

In [30]:
df['TRANSACTION_TYPE'] = df.apply(lambda row: get_transaction_type(row['type']), axis=1) # TODO - fix this!

C:\Users\dshorstein\AppData\Local\Continuum\Anaconda3\envs\usa-spent\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
df.TRANSATION_TYPE

AttributeError: 'DataFrame' object has no attribute 'TRANSATION_TYPE'